In [1]:
#@title Colab setup

import os, sys, subprocess
if "google.colab" in sys.modules:
  cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic"
  process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
# ------

import scipy.integrate
import bokeh.io
import numpy as np

import bokeh.io
import bokeh.plotting
from bokeh.models import LinearColorMapper, ColorBar

bokeh.io.output_notebook()

In [2]:
#@title Auxiliary functions

# Function to do the 2D projection of decision boundaries with normalization
def contourf(p, x, y, z, title=None, palette="Spectral11", normal = True):
    """Make a filled contour plot given x, y, z data given in 2D arrays."""

    # Normalize the z values
    if normal:
      z_min = z.min()
      z_max = z.max()
      z_normalized = (z - z_min) / (z_max - z_min)  # Normalize to range [0, 1]
    else:
      z_normalized = z

    # Add zero padding at the boundaries for better visualization
    N = z_normalized.shape[1]
    z0 = np.c_[z_normalized, np.zeros(N)]
    z0[-1, -1] = 1.  # Ensure the padding contains a value within range

    # Plot the normalized values
    p.image(
        image=[z0],
        x=x.min(),
        y=y.min(),
        dw=(x.max() - x.min()) * (1 + 1 / N),
        dh=x.max() - x.min(),
        palette=palette,
        alpha=0.8,
    )

    # Color mapping based on the normalized z0 values
    color = LinearColorMapper(palette=palette, low=z0.min(), high=z0.max())
    cb = ColorBar(color_mapper=color, location=(0, 0), width=10)
    p.add_layout(cb, 'right')

    return ()

We model the sequestration-based networks with the following chemical reactions proposed by [Moorman (2019)](https://ieeexplore.ieee.org/document/9030122)

\begin{align}
X_1 \xrightarrow{w_1} X_1 + Y \\
X_2 \xrightarrow{w_2} X_2 + Z\\
Y + Z \xrightarrow{\gamma} C
\end{align}

The corresponding reaction rate equations are given by the following ODEs:

\begin{eqnarray}
\frac{dy}{dt} &=& w_1 x_1 + \delta y - \gamma y z  \\
\frac{dz}{dt} &=& w_2 x_2 + \delta c - \gamma y z  \\
\end{eqnarray}

where $\gamma$ is the sequestration rate.

### Linear classification

In [3]:
# Definition of the chemical reactions
def ODE_Seq(X, t, a, b, d, g):

  y, z = X

  return (
      [
          a  - d*y - g*y*z,
          b - d*z - g*y*z,
      ]
  )

In [13]:
#@title Activation function and perceptron

# Set up figures
fig_size = [250, 225]
plots = []
for i in range(2): # the number of plots
  plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          #x_range=(-1, 1), #y_range=(0, 1),
                         ))
  plots[i].axis.major_label_text_font_size = '10pt'

# Color palette
colors = bokeh.palettes.grey(4)

# Sequestration reaction parameters
t = np.linspace(0,10,100)
av = np.linspace(0, 1, 50) # remember a = w1*x1
b = 0.5                    # remember b = w2*x2
g = 1000
d = 1

# Solving the ODEs
output = np.zeros(50)
for i, ai in enumerate(av):
  y_sol = scipy.integrate.odeint(ODE_Seq, [0., 0.], t, args=(ai, b, d, g))
  output[i] = y_sol[-1,0]
plots[0].line(av, output, line_width=3, color=colors[0])

# Computing a linear classifier
N = 30 # resolution
x1 = np.linspace(0,1,N) # [uM]
x2 = np.linspace(0,1,N) # [uM]
w1 = 1 # h^-1
w2 = 1 # h^-1

# Set up simulations
output = np.zeros((N,N))

for i, x1i in enumerate(x1):
    for j, x2j in enumerate(x2):
        y_sol = scipy.integrate.odeint(ODE_Seq, [0., 0.,], t, args=(x1i*w1, x2j*w2, d, g))
        output[j,i] = y_sol[-1,0] # we get the steady-state value

# Visualizing decision boundary
palette = bokeh.palettes.grey(9)[::-1]
contourf(plots[1], x1, x2, output, palette=palette, normal = True)

# Tidy up and set export settings
plots[0].title.text = 'Softplus-like'
plots[0].xaxis.axis_label = 'x1w1 - x2w2'
plots[0].yaxis.axis_label = 'y'
plots[1].title.text = 'Linear classification'
plots[1].xaxis.axis_label = 'x1'
plots[1].yaxis.axis_label = 'x2'
plots[1].width = 320
plots[1].x_range = bokeh.models.Range1d(x1[0],x1[-1])
plots[1].y_range = bokeh.models.Range1d(x2[0],x2[-1])

for i in range(2):
  plots[i].output_backend = 'svg'
bokeh.io.show(bokeh.layouts.row(plots))

### Non-linear classification

In [16]:
def nonlinear_classifier(X, t, x1, x2, w_hidden, w_output, d, g):

    # Unpacking the weights
    w1_0, w1_1, w1_2 = w_hidden[0]  # First (hidden) node weights
    w2_0, w2_1, w2_2 = w_hidden[1]  # Second (hidden) node weights
    w3_0, w3_1, w3_2 = w_output     # Output node weights

    # Hidden layer (left side of ODEs): n1 = Node 1, n2 = Node 2
    z1_n1, z2_n1, z1_n2, z2_n2, z1, z2 = X

    # Hidden layer (right side of ODEs)
    dz1_n1_dt = w1_0 + x2*w1_2 - d*z1_n1 - g*z1_n1*z2_n1
    dz2_n1_dt = x1*w1_1 - d*z2_n1 - g*z1_n1*z2_n1

    dz1_n2_dt = w2_0 + x1*w2_1 - d*z1_n2 - g*z1_n2*z2_n2
    dz2_n2_dt = x2*w2_2 - d*z2_n2 - g*z1_n2*z2_n2

    # Output node
    a = z1_n1*w3_1 + z1_n2*w3_2
    b = w3_0
    dz1_dt = a - d*z1 - g*z1*z2
    dz2_dt = b - d*z2 - g*z1*z2

    return [dz1_n1_dt, dz2_n1_dt, dz1_n2_dt, dz2_n2_dt, dz1_dt, dz2_dt]

In [18]:
#@title Multi-layer perceptron
# First Node's weights
w1_0 = 0.3
w1_1 = 0.5
w1_2 = 0.2

# Second node's weights
w2_0 = 0.2
w2_1 = 0.2
w2_2 = 0.5

# Third node (output)
w3_0 = 0.3
w3_1 = 0.3
w3_2 = 0.3

# Set up figures
fig_size = [300, 225]
plots = []
for i in range(3):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1]))
    plots[i].axis.major_label_text_font_size = '10pt'

# Simulation parameters
t = np.linspace(0,10,100) # 10 hours
g = 100 # sequestration rate
d = 1 # degradation rate

# Input space
N = 40 # resolution
x1 = np.linspace(0,1,N) # [uM]
x2 = np.linspace(0,1,N) # [uM]

# Seting up simulations
output1 = np.zeros((N,N))
output2 = np.zeros((N,N))
output3 = np.zeros((N,N))

# Organizing weights into arrays
w_hidden = np.array([
    [w1_0, w1_1, w1_2],  # First hidden node (w1_0, w1_1, w1_2)
    [w2_0, w2_1, w2_2]   # Second hidden node (w2_0, w2_1, w2_2)
])

w_output = np.array([w3_0, w3_1, w3_2])  # Output node weights (w3_0, w3_1, w3_2)

# Solving ODEs
for i, x1i in enumerate(x1):
    for j, x2j in enumerate(x2):
        # Solve the full system at once
        y_sol = scipy.integrate.odeint(nonlinear_classifier,
                                       [0., 0., 0., 0., 0., 0.], t,
                                       args=(x1i, x2j, w_hidden, w_output, d, g))

        # We are interested in steady-state values
        output1[j, i] = y_sol[-1, 0]  # Output of first node
        output2[j, i] = y_sol[-1, 2]  # Output of second node
        output3[j, i] = y_sol[-1, 4]  # Output of final node

# Visualizing decision boundary
palette = bokeh.palettes.Greys[9][::-1]
contourf(plots[0], x1, x2, output1, palette=palette, normal=True)
contourf(plots[1], x1, x2, output2, palette=palette, normal=True)
contourf(plots[2], x1, x2, output3, palette=palette, normal=True)

# Tidy up
for i in range(3):
    plots[i].x_range = bokeh.models.Range1d(x1[0], x1[-1])
    plots[i].y_range = bokeh.models.Range1d(x2[0], x2[-1])
    plots[i].output_backend = 'svg'

plots[0].title.text = '(Hidden) Node 1. Linear'
plots[1].title.text = ' (Hidden) Node 2. Linear'
plots[2].title.text = '(Output) Node 3. Nonlinear'

bokeh.io.show(bokeh.layouts.row(plots))

## Phosphorylation-based neural networks

We model the sequestration-based networks with the following chemical reactions proposed by [Cuba Samaniego (2021)](https://ieeexplore.ieee.org/document/9482800)

\begin{align}
\emptyset \xrightarrow{u} Z \\
\emptyset \xrightarrow{v} Y\\
∅ \xrightarrow{\theta} P^*
\end{align}

where $Z$ is a kinase, $Y$ is a phosphatase, and $P^*$ is the target protein. The phosphorylation reaction is given by the following chemical reaction:

\begin{align}
  Z + P^* \longleftrightarrow^{a_1}_{d_1} C_1 \xrightarrow{k_1} Z + P
\end{align}

Similarly, the dephosphorylation via the phosphatase is given by the following chemical reaction:

\begin{align}
  Y + P \longleftrightarrow^{a_2}_{d_2} C_2 \xrightarrow{k_2} Y + P^*
\end{align}

Considering the law of mass action: $z^T = z + c_1$, $y^T = y + c_2$ and $p^T = p + p^* + c_1 + c_2$, and under quasi steady-state assumptions for $\dot c_1 \approx 0$ and $\dot c_2 \approx 0$, we can model the chemical reactions with the following ODEs

\begin{eqnarray}
  \frac{dz^T}{dt} &=& u - \delta z^T \\
  \frac{dy^T}{dt} &=& v - \delta y^T \\
  \frac{dp^T}{dt} &=& \theta - \delta p^T \\
  \frac{dp}{dt} &=& k_1 \frac{p^T - p}{p^T - p + K_1}z^T - k_2 \frac{p}{p + K_2}y^T
\end{eqnarray}


In [19]:
def ODE_Sig(X, t, u, v, d, th, K1, K2, k1, k2):

  zT, yT, pT, p = X

  return (
      [
          u - d*zT,
          v - d*yT,
          th - d*pT,
          k1 * (pT - p) / (pT - p + K1) * zT - k2 * p / (p + K2) * yT
      ]
  )

In [23]:
#@title Activation function and perceptron

# Set up figures
fig_size = [250, 225]
plots = []
for i in range(2): # the number of plots
  plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          #x_range=(-1, 1), #y_range=(0, 1),
                         ))
  plots[i].axis.major_label_text_font_size = '10pt'

# Color palette
colors = bokeh.palettes.grey(4)

# Phosphorylation/dephosphorylation reaction parameters
t = np.linspace(0,10,100)
uv = np.linspace(0, 1, 50)
v = 0.5
d = 1
th = 1
k1 = 10
k2 = 10
K1 = 0.05
K2 = 0.05

# Solving the ODEs
output = np.zeros(50)
for i, ui in enumerate(uv):
  y_sol = scipy.integrate.odeint(ODE_Sig, [0., 0., 0., 0.], t, args=(ui, v, d, th, K1, K2, k1, k2))
  output[i] = y_sol[-1,3]
plots[0].line(av, output, line_width=3, color=colors[0])

# Computing a linear classifier
N = 30 # resolution
x1 = np.linspace(0,1,N) # [uM]
x2 = np.linspace(0,1,N) # [uM]
w1 = 1 # h^-1
w2 = 1 # h^-1

# Set up simulations
output = np.zeros((N,N))

for i, x1i in enumerate(x1):
    for j, x2j in enumerate(x2):
        y_sol = scipy.integrate.odeint(ODE_Sig, [0., 0., 0., 0.], t, args=(x1i*w1, x2j*w2, d, th, K1, K2, k1, k2))
        output[j,i] = y_sol[-1,3] # we get the steady-state value

# Visualizing decision boundary
palette = bokeh.palettes.grey(9)[::-1]
contourf(plots[1], x1, x2, output, palette=palette, normal = True)

# Tidy up and set export settings
plots[0].title.text = 'Sigmoid-like'
plots[0].xaxis.axis_label = 'x1w1 - x2w2'
plots[0].yaxis.axis_label = 'p'
plots[1].title.text = 'Linear classification'
plots[1].xaxis.axis_label = 'x1'
plots[1].yaxis.axis_label = 'x2'
plots[1].width = 320
plots[1].x_range = bokeh.models.Range1d(x1[0],x1[-1])
plots[1].y_range = bokeh.models.Range1d(x2[0],x2[-1])

for i in range(2):
  plots[i].output_backend = 'svg'
bokeh.io.show(bokeh.layouts.row(plots))